
polygraphy를 설치하기 위해서는 다음의 명령어를 사용해야한다.

`python -m pip install colored polygraphy --extra-index-url https://pypi.ngc.nvidia.com`

polygraphy는 의존하는 패키지가 많이 없기때문에 아래에 `AUTOINSTALL_DEPS`를 True로 바꿔주면 알아서 설치가된다.

In [1]:
import polygraphy
polygraphy.config.AUTOINSTALL_DEPS = True
polygraphy.config.ASK_BEFORE_INSTALL = True # 지 멋대로 설치되는 것을 방지하기 위해서
print(f"{polygraphy.config.AUTOINSTALL_DEPS=}")

polygraphy.config.AUTOINSTALL_DEPS=True


# Backends

`Bankends`는 deep learning framework와의 interface를 제공한다. Backends는 Loader와 Runner로 구성되어있다.

일단 Loader에 대해서 먼저 알아보자.
Polygraphy는 두 가지 Loader를 제공한다. `PascalCase`의 경우 TensorRT엔진을 만들 수 있는 callable을 반환하고, `snakecase`의 경우 callable이 아니라 engine을 반환한다.
즉 전자를 사용하면 나중에 호출할 때 engine이 생성되고, 후자를 사용하면 바로 engine이 생성된다.

In [2]:
from polygraphy.backend.trt import EngineFromNetwork, NetworkFromOnnxPath

build_engine = EngineFromNetwork(NetworkFromOnnxPath("/home/park/ros2/tensorRT/policy.onnx"))
print(type(build_engine))

[I] TF32 is disabled by default. Turn on TF32 for better performance with minor accuracy differences.
<class 'polygraphy.backend.trt.loader.EngineFromNetwork'>


In [3]:
from polygraphy.backend.trt import engine_from_network, network_from_onnx_path

engine = engine_from_network(network_from_onnx_path("/home/park/ros2/tensorRT/policy.onnx"))
print(type(engine))

[I] TF32 is disabled by default. Turn on TF32 for better performance with minor accuracy differences.
[I] Configuring with profiles:[
        Profile 0:
            {qpos [min=[1, 6], opt=[1, 6], max=[1, 6]],
             image [min=[1, 3, 3, 480, 640], opt=[1, 3, 3, 480, 640], max=[1, 3, 3, 480, 640]]}
    ]
[W] profileSharing0806 is on by default in TensorRT 10.0. This flag is deprecated and has no effect.
[I] Building engine with configuration:
    Flags                  | []
    Engine Capability      | EngineCapability.STANDARD
    Memory Pools           | [WORKSPACE: 11783.31 MiB, TACTIC_DRAM: 11783.31 MiB, TACTIC_SHARED_MEMORY: 1024.00 MiB]
    Tactic Sources         | [EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [PROFILE_SHARING_0806]
[I] Finished engine building in 14.640 seconds
<class 'tensorrt.tensorrt.ICudaEngine'>


# Runner

Runner는 Loader를 사용해 모델을 로드하고, 추론을 실행하는 객체이다.
Runner를 사용하기 위해서는 activate를 해야하는데 한 번 activate하는데 비용이 크므로, 여러번 하지 않는 것이 좋다. 그리고 Context Manager를 사용하는 것을 권장한다.

In [4]:
from polygraphy.backend.trt import TrtRunner

with TrtRunner(build_engine) as runner:
    outputs = runner.infer(feed_dict={"input0": input_data})

[I] Configuring with profiles:[
        Profile 0:
            {qpos [min=[1, 6], opt=[1, 6], max=[1, 6]],
             image [min=[1, 3, 3, 480, 640], opt=[1, 3, 3, 480, 640], max=[1, 3, 3, 480, 640]]}
    ]
[I] Building engine with configuration:
    Flags                  | []
    Engine Capability      | EngineCapability.STANDARD
    Memory Pools           | [WORKSPACE: 11783.31 MiB, TACTIC_DRAM: 11783.31 MiB, TACTIC_SHARED_MEMORY: 1024.00 MiB]
    Tactic Sources         | [EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [PROFILE_SHARING_0806]
[I] Finished engine building in 14.286 seconds


NameError: name 'input_data' is not defined

# Comparator

## ONNX Model Export

ONNX Model을 Export하기 위해서는 `torch.onnx.export`함수를 사용해야한다. 이 함수를 사용할 때 dummy input을 넣어줘야하는데 ACT신경망의 경우, 학습과 추론시에 신경망이 다르다. 따라서 추론시에 어떤 신경망을 사용하는지 살펴보고, 어떻게 dummy input을 구성해야하는지 알아보자. 아래 코드에서 policy가 ACT신경망을 말하는데 여기에 입력이 qpos와 curr_image임을 알 수 있다. 저희 로봇 같은 경우는 6개의 관절이 있으므로 qpos는 shpae이 (6,), 3대의 카메라가 있으므로 shape이 (3,480,640,3)입니다.

아래 코드는 ACT에서 사용된 ACTPolicy이다. `__call__`이 호출되었을 때 actions가 있으면 training을 진행하고, 없으면 Inference만 진행한다.

In [ ]:
class ACTPolicy(nn.Module):
    def __init__(self, args_override):
        super().__init__()
        model, optimizer = build_ACT_model_and_optimizer(args_override)
        self.model = model # CVAE decoder
        self.optimizer = optimizer
        self.kl_weight = args_override['kl_weight']
        print(f'KL Weight {self.kl_weight}')

    def __call__(self, qpos, image, actions=None, is_pad=None):
        env_state = None
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        image = normalize(image)
        if actions is not None: # training time
            actions = actions[:, :self.model.num_queries]
            is_pad = is_pad[:, :self.model.num_queries]

            a_hat, is_pad_hat, (mu, logvar) = self.model(qpos, image, env_state, actions, is_pad)
            total_kld, dim_wise_kld, mean_kld = kl_divergence(mu, logvar)
            loss_dict = dict()
            all_l1 = F.l1_loss(actions, a_hat, reduction='none')
            l1 = (all_l1 * ~is_pad.unsqueeze(-1)).mean()
            loss_dict['l1'] = l1
            loss_dict['kl'] = total_kld[0]
            loss_dict['loss'] = loss_dict['l1'] + loss_dict['kl'] * self.kl_weight
            return loss_dict
        else: # inference time
            a_hat, _, (_, _) = self.model(qpos, image, env_state) # no action, sample from prior
            return a_hat

In [ ]:
if config['policy_class'] == "ACT":
    if t % query_frequency == 0:
        all_actions = policy(qpos, curr_image)

아래는 제가 작성한 ONNX모델 출력코드입니다. qpos, curr_image의 shape에서 batch size만 추가해서 작성해주면 됩니다.

In [ ]:
batch_size = 1
state_dim = 14
num_cameras = len(camera_names)
channels = 3
height = 480
width = 640

dummy_qpos = torch.randn(batch_size, state_dim).cuda()
dummy_image = torch.randn(batch_size, num_cameras, channels, height, width).cuda()

# ONNX로 export
onnx_path = os.path.join(ckpt_dir, "policy.onnx")
torch.onnx.export(
    policy,                                 # 변환할 모델
    (dummy_qpos, dummy_image),              # 입력 튜플
    onnx_path,                              # 저장 경로
    export_params=True,                     # 모델 파라미터 저장
    opset_version=17,                       # ONNX opset 버전
    do_constant_folding=True,               # 상수 폴딩 최적화
    input_names=['qpos', 'image'],          # 입력 이름
    output_names=['action'],                # 출력 이름 (모델에 따라 다를 수 있음)
    dynamic_axes={                          # 배치 크기 등 동적 처리
        'qpos': {0: 'batch_size'},
        'image': {0: 'batch_size'}
    }
)
print(f"ONNX 모델이 {onnx_path}에 저장되었습니다.")

# Calibrator

Calibrator는 INT8추론을 위해 네트워크를 생성할 때 FP32값을 양자화할 때 사용해야하는 파라미터를 계산해주는 클래스이다.
`Calibrator(data_loader, cache=None, BaseClass=None, batch_size=None, quantile=None, regression_cutoff=None, algo=None)[source]`
Calibrator에는 `data_loader`라는 iterable 혹은 generator를 인자로 전달해줘야한다. 이 `data_loader`를 만들기 위해서 ACT에서 데이터를 어떻게 전처리하고, 어떤 데이터를 가져오는지를 분석할 필요가 있다.

해당 내용은 act/utils.py에 있는 `Episodic dataset`과 `get_norm_stats`에 있다. 아래 코드를 보면, 모든 에피소드의 qpos, qvel, action을 numpy array에 저장한 후에, 평균과 표준편차를 계산한 뒤 이를 반환한다. 이 데이터는 데이터를 정규화할 때 사용된다.

In [ ]:
def get_norm_stats(dataset_dir, num_episodes):
    all_qpos_data = []
    all_action_data = []
    for episode_idx in range(num_episodes):
        dataset_path = os.path.join(dataset_dir, f'episode_{episode_idx}.hdf5')
        with h5py.File(dataset_path, 'r') as root:
            qpos   = root['/observations/qpos'][()]
            qvel   = root['/observations/qvel'][()]
            action = root['/action'][()]
        all_qpos_data.append(torch.from_numpy(qpos))
        all_action_data.append(torch.from_numpy(action))
    all_qpos_data   = torch.stack(all_qpos_data)   # shape: [N, T, qpos_dim]
    all_action_data = torch.stack(all_action_data) # shape: [N, T, action_dim]

    # (1) action 데이터 정규화 통계
    action_mean = all_action_data.mean(dim=[0,1], keepdim=True)
    action_std  = all_action_data.std(dim=[0,1], keepdim=True)
    action_std  = torch.clip(action_std, 1e-2, np.inf)

    # (2) qpos 데이터 정규화 통계
    qpos_mean = all_qpos_data.mean(dim=[0,1], keepdim=True)
    qpos_std  = all_qpos_data.std(dim=[0,1], keepdim=True)
    qpos_std  = torch.clip(qpos_std, 1e-2, np.inf)

    stats = {
      "action_mean":  action_mean.numpy().squeeze(),
      "action_std":   action_std.numpy().squeeze(),
      "qpos_mean":    qpos_mean.numpy().squeeze(),
      "qpos_std":     qpos_std.numpy().squeeze(),
      "example_qpos": qpos  # 마지막 에피소드의 raw qpos 예시
    }
    return stats


아래 EpisodicDataset을 보면, `__getitem__`함수에서 에피소드의 특정순간(start_ts)의 `qpos`, `qvel`, `image`들고온 후, start_ts이후에 `action`값을 들고온다.
학습 시킬 때, `action sequnce`가 필요하고, 에피소드의 끝일 경우 `action sequnce`의 개수가 부족하기때문에 padding을 수행한 후, `is_pad`변수를 통해 paddding이 되어있는지 여부를 저장한다. 이미지는 `k c h w`형태로 변경하고, image, action , qpos를 정규화한다.

정리하자면,
1. 특정 index의 `qpos`, `image`, `action`을 저장한다. 이때 `action`은 `qpos`의 index이후의 값을 **전부** 저장한다.
2. `action`은 padding을 하고, `is_pad`를 통해 padding 여부를 저장한다.
3. `qpos`, `image`, `action`를 정규화한다.

ACT신경망은 학습할때와 추론할때 사용되는 신경망이 다르다. 학습할 때는 `qpos`, `image`, `action` 모두 필요하지만, 추론할 때는 `qpos`, `image`만 있으면 된다. 따라서, action은 input으로 넣어주지 않아도 된다.

이렇게 Pytorch Custom Dataset을 구현하고나면 데이터를 한 개씩 꺼낼 수 있는 


In [ ]:
class EpisodicDataset(torch.utils.data.Dataset):
    def __init__(self, episode_ids, dataset_dir, camera_names, norm_stats):
        super(EpisodicDataset).__init__()
        self.episode_ids = episode_ids
        self.dataset_dir = dataset_dir
        self.camera_names = camera_names
        self.norm_stats = norm_stats
        self.is_sim = None
        self.__getitem__(0) # initialize self.is_sim

    def __len__(self):
        return len(self.episode_ids)

    def __getitem__(self, index):
        sample_full_episode = False # hardcode

        episode_id = self.episode_ids[index]
        dataset_path = os.path.join(self.dataset_dir, f'episode_{episode_id}.hdf5')
        with h5py.File(dataset_path, 'r') as root:
            is_sim = root.attrs['sim']
            original_action_shape = root['/action'].shape # shape : (timesteps, 6) -> (20s x 30hz, 6)
            episode_len = original_action_shape[0]
            if sample_full_episode:
                start_ts = 0
            else:
                start_ts = np.random.choice(episode_len)
            # get observation at start_ts only
            qpos = root['/observations/qpos'][start_ts] # shape : (qpos,)
            qvel = root['/observations/qvel'][start_ts]
            image_dict = dict()
            for cam_name in self.camera_names:
                image_dict[cam_name] = root[f'/observations/images/{cam_name}'][start_ts] # shape : (480, 640, 3)
            # get all actions after and including start_ts
            if is_sim:
                action = root['/action'][start_ts:] # shape : (action_len, 6)
                action_len = episode_len - start_ts
            else:
                action = root['/action'][max(0, start_ts - 1):] # hack, to make timesteps more aligned
                action_len = episode_len - max(0, start_ts - 1) # hack, to make timesteps more aligned

        self.is_sim = is_sim
        padded_action = np.zeros(original_action_shape, dtype=np.float32)
        padded_action[:action_len] = action
        is_pad = np.zeros(episode_len)
        is_pad[action_len:] = 1

        # new axis for different cameras
        all_cam_images = []
        for cam_name in self.camera_names:
            all_cam_images.append(image_dict[cam_name])
        all_cam_images = np.stack(all_cam_images, axis=0) # shape : (3, 480, 640 ,3)

        # construct observations
        image_data = torch.from_numpy(all_cam_images)
        qpos_data = torch.from_numpy(qpos).float()
        action_data = torch.from_numpy(padded_action).float()
        is_pad = torch.from_numpy(is_pad).bool()

        # channel last
        image_data = torch.einsum('k h w c -> k c h w', image_data) # shape : (3, 3, 480, 640)

        # normalize image and change dtype to float
        image_data = image_data / 255.0
        action_data = (action_data - self.norm_stats["action_mean"]) / self.norm_stats["action_std"]
        qpos_data = (qpos_data - self.norm_stats["qpos_mean"]) / self.norm_stats["qpos_std"]

        return image_data, qpos_data, action_data, is_pad

이렇게 Pytorch Custom Dataset을 구현하고나면 데이터를 한 개씩 꺼낼 수 있는 준비는 마친셈이다. train_dataset, val_dataset을 구축하고, dataloader를 통해 해당 데이터들에 접근이 가능하다. 

In [ ]:
from utils import EpisodicDataset, load_data, get_norm_stats

dataset_dir = "/home/park/ros2/tensorRT/meloha_box_picking_data"
camera_names = ['cam_head', 'cam_left_wrist','cam_right_wrist']
batch_size_train = 8
batch_size_val = 8

train_dataloader, val_dataloader, stats, _ = load_data(dataset_dir,
                                                        41,
                                                        camera_names,
                                                        batch_size_train,
                                                        batch_size_val
                                                        )

image_data, qpos_data, action_data, is_pad = next(iter(train_dataloader))
print(f"{qpos_data.shape=}") # qpos_data.shape=torch.Size([8, 6])
print(f"{action_data.shape=}") # action_data.shape=torch.Size([8, 600, 6])
print(f"{image_data.shape=}") # image_data.shape=torch.Size([8, 3, 3, 480, 640])


하지만, [Calibrator](https://docs.nvidia.com/deeplearning/tensorrt/archives/tensorrt-1001/polygraphy/docs/backend/trt/calibrator.html?utm_source=chatgpt.com)를 참고해보면 ONNX모델을 만들 때 처럼 입력 이름과 입력 값을 딕셔너리형태로 전달해줘야한다. 값은 Numpy, Pytorch Tensor, GPU Pointers가 가능하다. 기존 코드에서 사용중인 `EpisodicDataset`은 단순히 shape이 (3, 3, 480, 640)인 image_data, shape이 (6,)인 qpos_data, shape이 (600,6)인 action_data, is_pad를 반환하고 있다.

# IBuilderConfig

저는 아래와 같이 `IBuilderConfig`를 작성하였습니다. INT8 양자화를 지원하지않는 레어이거 존재하면 FP32로 계산됩니다. 이때, FP16으로 계산을 원한다면 `fp16=True` 매개변수를 설정해주면 됩니다.

In [ ]:
# Each type flag must be set to true.
builder_config = poly_trt.create_config(builder=builder,
                                        network=network,
                                        int8=True,
                                        fp16=True,
                                        calibrator=calibrator)

# TensorRT Engine Build and Save

In [1]:
engine = poly_trt.engine_from_network(network=(builder, network, parser),
                                      config=builder_config)

# TensorRT engine will be saved to ENGINE_PATH.
ENGINE_PATH = "/home/park/ros2/tensorRT/act_int8_fp16.engine"
poly_trt.save_engine(engine, ENGINE_PATH)

# Load serialized engine using 'open'.
engine = poly_trt.engine_from_bytes(open(ENGINE_PATH, "rb").read())
print("engine이 저장되었습니다.")

NameError: name 'poly_trt' is not defined

따라서, 엔진을 생성하기 전에 가장 먼저 생성해야하는 클래스이다.
엔진은 `network_from_onnx_path`와 `engine_from_network`를 통해 생성이 가능하다. 둘 다 `from polygraphy.backend.trt`안에 들어있는 함수이다.
`network_from_onnx_path`는 ONNX모델의 경로를 받아 TensorRT network, builder, parser를 반환한다. (!network, builder, parser가 정확히 뭔지 모르겠다)!

builder와 network를 통해 `IBuilderconfig`를 생성해야한다.
`IBuilderConfig` 객체를 만들 때는 정수 타입을 지원하지 않는 레이어를 위해 FP16 타입 변환에 대한 옵션을 추가해야 합니다. 여기서 주의할 점은, `IBuilderConfig`은 각 타입에 대한 인자를 해당 타입 사용 여부를 나타내는 플래그로 사용한다는 것입니다. 따라서, INT8과 FP16 타입을 모두 사용하려면 `create_config`에서 이들 타입 각각에 해당되는 인자를 모두 `True`로 설정해야 합니다.
마지막으로, 앞서 만든 `Calibrator` 객체를 인자로 추가하면 `IBuilderConfig` 객체 생성이 완료됩니다.

저 같은 경우 다음과 같은 Warning이 발생했습니다.
[W] Missing scale and zero-point for tensor ONNXTRT_Broadcast_1182_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model/backbones.0/backbones.0.1/Constant_1_output_0_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model.transformer.encoder.layers.3.norm1.weight_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model.transformer.decoder.layers.0.norm2.weight_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model/backbones.0/backbones.0.1_2/Constant_output_0_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor

위 경고중에서 `ONNXTRT_Broadcast_1182_output`는 왜 발생했는지 찾지못했습니다.
그리고 다른 경고들은 TensorRT에서 LayerNorm의 양자화를 지원하지 않기때문에 발생하는 경고입니다. 그리고 `model/backbones.0/backbones.0.1_2/Constant_output_0`는 ACT가 `detr`을 사용하는데 Position Encoding을 할 때 일부 레이어에서 양자화를 지원하지 않기때문에 발생하는 경고입니다.

# Engine 성능 비교

In [ ]:
Engine의 성능을 비교하기 위해 다음 세 가지 타입의 Engine을 생성합니다.
1. FP32 원본 엔진(No Calibrator)
2. INT8 Static Calibration 적용된 엔진
3. INT8 Random Calibration 적용된 엔진(Calibration으로 인한 정확도 향상을 알아보기위해)